<a href="https://colab.research.google.com/github/Saritakumari540/Depression-detection-by-audio/blob/main/DepressionDetectionByAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import dependencies

In [2]:
!wget https://zenodo.org/records/1188976/files/Audio_Speech_Actors_01-24.zip?download=1

--2024-11-09 21:10:27--  https://zenodo.org/records/1188976/files/Audio_Speech_Actors_01-24.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208468073 (199M) [application/octet-stream]
Saving to: ‘Audio_Speech_Actors_01-24.zip?download=1’

Audio_Speech_Actors 100%[===================>] 198.81M  30.0MB/s    in 7.5s    

2024-11-09 21:10:35 (26.4 MB/s) - ‘Audio_Speech_Actors_01-24.zip?download=1’ saved [208468073/208468073]



In [8]:
!mkdir -p data/input
!mkdir -p data/working/Depression
!mkdir -p data/working/NonDepression

In [ ]:
!unzip Audio_Speech_Actors_01-24.zip?download=1 -d ./data/input

In [9]:
!pip install tensorflow-io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 13.4 MB/s eta 0:00:00


In [11]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import shutil

# Data Preparation

In [14]:
base_source_directory = "data/input"
destination_directories = {
    'Depression': 'data/working/Depression/',
    'NonDepression': 'data/working/NonDepression/'
}

def copy_files_with_specific_number():
    count = 0
    for i in range(1, 25):
        actor_folder = f"Actor_{str(i).zfill(2)}"
        source_directory = os.path.join(base_source_directory, actor_folder)

        for category, base_destination_directory in destination_directories.items():
            destination_directory = os.path.join(base_destination_directory, actor_folder)

            if not os.path.exists(destination_directory):
                os.makedirs(destination_directory)

            for filename in os.listdir(source_directory):
                if '-' in filename:
                    parts = filename.split('-')
                    if len(parts) >= 3:
                        number = parts[2][:2]
                        if (category == 'Depression' and number in ['04', '05', '06', '07']) or \
                           (category == 'NonDepression' and number in ['01', '02', '03', '08']):
                            source_path = os.path.join(source_directory, filename)
                            destination_path = os.path.join(destination_directory, filename)
                            shutil.copy(source_path, destination_path)
                            count += 1
    print(f"Total files moved: {count}")

copy_files_with_specific_number()

Total files moved: 1440
